In [1]:
%%capture
try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain
packages = ['twython', 'pandas', 'psycopg2-binary', 'twint', 'nest_asyncio']
pipmain(['install'] + packages)

In [2]:
import twint
import pandas as pd
import nest_asyncio
import datetime
nest_asyncio.apply()

In [3]:
%%capture
import sys
sys.path.append('../')
from get_tweet_data import *

from aws import host as ahost, port as aport, user as auser, password as apassword, database as adatabase
import psycopg2
connection = psycopg2.connect(host = ahost, 
                              port = aport, 
                              user = auser, 
                              password = apassword, 
                              dbname = adatabase)
cursor = connection.cursor()

In [50]:
FOLLOWERS_LIMIT = 20
MIN_WORD_LEN = 3

In [5]:
twitter_handles = ['realDonaldTrump','ewarren','JoeBiden', 'SenSanders','KamalaHarris','CoryBooker','AndrewYang']
def get_followers_list(username, limit = 200):
    c = twint.Config()
    c.Limit = limit
    c.Hide_output = True
    c.Username = username
    c.Pandas = True

    twint.run.Followers(c)
    
    f = twint.storage.panda.Follow_df
    list_of_followers = f['followers'][username]
    
    return list_of_followers

In [7]:
for user in twitter_handles:
    followers = get_followers_list(user, limit = FOLLOWERS_LIMIT)
    write_user_followers_to_db(followers, user)
    #print(read_user_followers_from_db(user)) #debug line

In [8]:
twitter_handles = ['realDonaldTrump','ewarren','JoeBiden', 'SenSanders','KamalaHarris','CoryBooker','AndrewYang']
user = twitter_handles[0]
followers = read_user_followers_from_db(user)

In [9]:
def get_tweets_df(follower, limit=100):
    c = twint.Config()
    c.Limit = limit
    c.Username = follower
    c.Pandas = True
    c.Retweets = False
    c.Hide_output = True  
    twint.run.Search(c)

    Tweets_df = twint.storage.panda.Tweets_df
    if len(Tweets_df) == 0:
        return pd.DataFrame()
    df = Tweets_df[['id', 'date', 'tweet', 'hashtags']]
    df.loc[:,'date'] = [datetime.datetime.strptime(s.split(' ')[0], '%Y-%m-%d') for s in df['date'].tolist()]
    df.loc[:,'id'] = df['id'].astype(int)
    return df

In [51]:
# noun based word extraction
def word_hashtag_extraction_noun_based(tweet_dictionary):  
    text = tweet_dictionary['tweet']
    text = casual_tokenize(tweet_dictionary['tweet'])
    result = nltk.pos_tag(text)
    
    all_noun_tags = ['NN', 'NNS', 'NNP', 'NNPS']
    proper_noun_tags = ['NNP', 'NNPS']
    
    cleaned_text = {}
    for t in result:
        word = t[0]
        tag = t[1]
        
        # words
        if (tag in all_noun_tags) and isEnglish(word) and (len(word) >= MIN_WORD_LEN):
            if tag in proper_noun_tags:
                cleaned_text[word.lower()] = PROPER_NOUN_WORD_TYPE 
            else:
                cleaned_text[word.lower()] = REGULAR_NOUN_WORD_TYPE
                
    hashtags = tweet_dictionary['hashtags']
    cleaned_tags = set()
    for h in hashtags:
        if isEnglish(h[1:]):
            cleaned_tags.add(h.lower())
    
    return cleaned_text, cleaned_tags

In [26]:
follower = followers['follower_handle'].tolist()[5]
print(follower)
df = get_tweets_df(follower)
print(df)

BugiacL
                    id       date  \
0  1198705113369591813 2019-11-24   

                                               tweet hashtags  
0  Hi, I am from România and I would be intereste...       []  


In [14]:
for i in range(len(df)):
    output = word_hashtag_extraction_noun_based(df.iloc[i,:])
    print(df['id'].iloc[i], output) #debug line

1198705113369591813 ({'hi': 1, 'model': 0}, set())


In [15]:
def write_tweets_words_hashtags_to_db(follower, df):
    for i in range(len(df)):
        tweet = df.iloc[i,:]
        if len(tweet) == 0:
            continue
        tweet_id = tweet['id']
        tweet_date = tweet['date']
        sql = 'INSERT INTO tweets VALUES ({},\'{}\',\'{}\');'.format(tweet_id, follower, tweet_date)

        # if writing tweet to database was sucessful, then we need to get write word count info
        if write_try(sql):
            words, tags = word_hashtag_extraction_noun_based(tweet)
            
            word_count = get_freq_map(words)
            hashtag_count = get_freq_map(tags)

            for w, c in word_count.items():
                word_type_int = words[w]
                sql2 = 'INSERT INTO words VALUES ({},\'{}\',{},{});'.format(tweet_id, w, c, word_type_int)
                write_try(sql2)

            for w, c in hashtag_count.items():
                sql2 = 'INSERT INTO hashtags VALUES ({},\'{}\',{});'.format(tweet_id, w, c)
                write_try(sql2)

In [27]:
print(follower)
write_tweets_words_hashtags_to_db(follower, df)

BugiacL


In [30]:
read_all_tweets_from_db()

,tweet_id,follower_handle,tweet_date
0,0,test_follower1,1977-01-08 04:05:06
1,1,test_follower1,1999-01-08 04:05:06
2,1198375711133884416,banks_mikado,2019-11-23 23:00:08
3,1198375272934002688,banks_mikado,2019-11-23 22:58:24
4,1198372948207448065,banks_mikado,2019-11-23 22:49:10
...,...,...,...
1989,1174069239302565888,kaba_kourtime,2019-09-17 00:00:00
1990,1153672126555205634,kaba_kourtime,2019-07-23 00:00:00
1991,1130577592258637825,kaba_kourtime,2019-05-20 00:00:00
1992,1130577174061359105,kaba_kourtime,2019-05-20 00:00:00


In [ ]:
for user in twitter_handles:
    followers = read_user_followers_from_db(user)
    #print(followers)
    for ind in followers.index: 
        f = followers['follower_handle'][ind]
        # print(f)
        df = get_tweets_df(f)
        write_tweets_words_hashtags_to_db(f, df)

In [41]:
def read_follower_tweets_from_db(follower):
    sql = ('SELECT tweet_date FROM tweets '
    'WHERE follower_handle = \'{}\''.format(follower)
    )
    return read_try(sql)

In [42]:
print(follower)
read_user_tweets_from_db(follower)

BugiacL


,tweet_date
0,2019-11-24


In [46]:
for ind in followers.index: 
    follower = followers['follower_handle'][ind]
    df = read_follower_tweets_from_db(follower)
    if len(df) > 0:
        print(follower)
        print(df.head())

Mark00574043
           tweet_date
0 2019-11-24 06:31:29
BugiacL
  tweet_date
0 2019-11-24
hugsey21
           tweet_date
0 2016-04-09 10:16:09
1 2016-04-04 16:49:24
Eilidh_baileys
           tweet_date
0 2019-11-19 08:18:35
1 2019-11-09 00:42:07
2 2019-11-02 08:27:44
3 2019-10-17 03:56:54
4 2019-10-11 02:23:45
FrankensteinNo3
           tweet_date
0 2019-11-22 21:14:35
1 2019-11-29 00:00:00
Cobra__King
           tweet_date
0 2019-11-24 09:12:13
1 2019-11-26 00:00:00
2 2019-11-26 00:00:00
3 2019-11-26 00:00:00
4 2019-11-26 00:00:00
coreyjoslin
           tweet_date
0 2017-10-31 19:21:24
CNEtalent
           tweet_date
0 2019-11-24 09:17:53
1 2019-11-24 09:17:50
2 2019-11-24 09:15:42
3 2019-11-24 09:15:19
4 2019-11-24 09:15:00
Sandeep76355350
  tweet_date
0 2019-11-25
philosophoon
  tweet_date
0 2019-06-26
1 2019-06-26
2 2019-04-02
3 2015-10-18
almarod67360237
  tweet_date
0 2019-11-26
1 2019-11-26
2 2019-11-25
3 2019-11-25
4 2019-11-25
FrankyyFrenchh
  tweet_date
0 2019-11-23
1 2019-1

In [44]:
followers

,follower_handle
0,JuniorPalesa
1,Mark00574043
2,delan_dee
3,meekreisy
4,LegendARne
...,...
460,ykxirob
461,issac00923042
462,MarkJor33573875
463,Gilbert87532677
